In [3]:
import pandas as pd
import math
import io
#from google.colab import files 
#import statsmodels.api as sm
import scipy
from scipy import stats
#from statsmodels.stats import weightstats as stests
#import numpy as np
import csv
from scipy.stats import ttest_ind
from pandas import DataFrame
import numpy as np
import time



In [4]:


#from google.colab import drive
#drive.mount('/content/drive')
#data = pd.read_csv('/content/data3k10cv.csv')
data = pd.read_csv('Newdata1800.csv')
#data = pd.read_csv('/content/drive/My Drive/OldData/Newdata1800.csv')
#data = pd.read_csv('testdatapairhypoN.csv')
dataOriginal=data.copy()


tempdfCFG=data.filter(like='CFG_').head(0)
tempdfPM=data.filter(like='PM_').head(0)
count_CFG=len(tempdfCFG.columns)
count_Per=len(tempdfPM.columns)
count_allcols=len(data.columns)



print("test indexing on columns")

print("count_cfg    :  ", count_CFG)
print("count_Per   :  ", count_Per)
print("count_Allcols : ", count_allcols)

stdzeroindex=[]

count_uniq=0
for i in range(count_CFG,count_allcols):
  #print('----',i,'--',data.iloc[:,i].name,'--')

  count_uniq=len(data.iloc[:,i].unique())  #check in perfomance columns
  if (count_uniq==1):
    print(data.iloc[:,i].name,data.iloc[:,i].unique())
    stdzeroindex.append(i)



count_uniq=0
for i in range(count_CFG):
  count_uniq=len(data.iloc[:,i].unique())
  if(count_uniq>15):
    print("A High Uniq item column",data.iloc[:,i].name,count_uniq)
    stdzeroindex.append(i)


data=data.fillna(-1)

data = data.drop(data.columns[stdzeroindex], axis=1)

tempdfCFG=data.filter(like='CFG_').head(0)
tempdfPM=data.filter(like='PM_').head(0)
count_CFG=len(tempdfCFG.columns)
count_Per=len(tempdfPM.columns)
count_allcols=len(data.columns)


print("count_cfg    :  ", count_CFG)
print("count_Per   :  ", count_Per)
print("count_Allcols : ", count_allcols)
print(data.shape)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (393,394,398,742,743,745,932,933,935,1366) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


test indexing on columns
count_cfg    :   1410
count_Per   :   21
count_Allcols :  1431
A High Uniq item column CFG_global_cellid 8193
A High Uniq item column CFG_EutranInterFreqNCell.eNodeBId 22
A High Uniq item column CFG_EutranInterFreqNCell.LocalCellName 50
A High Uniq item column CFG_EutranInterFreqNCell.NeighbourCellName 21
A High Uniq item column CFG_eUCommCellSectorEqm.eNodeBId 2144
A High Uniq item column CFG_eUCommCellSectorEqm.SectorEqmId 90
A High Uniq item column CFG_eUCellSectorEqm.SectorEqmId 90
A High Uniq item column CFG_PDSCHCfg.ReferenceSignalPwr 23
A High Uniq item column CFG_UtranNCell.RncId 19
A High Uniq item column CFG_UtranNCell.NCellMeasPriority 57
A High Uniq item column CFG_UtranNCell.LocalCellName 449
A High Uniq item column CFG_UtranNCell.NeighbourCellName 74
A High Uniq item column CFG_UtranNCell.NCellAdditionTime 81
A High Uniq item column CFG_Cell.CellName 8165
A High Uniq item column CFG_Cell.PhyCellId 466
A High Uniq item column CFG_Cell.RootSequenceI

In [2]:
pernameslist=list(tempdfPM.columns)

combidataA=data.groupby([data.iloc[:,0].name],as_index=False)[pernameslist].agg(['count','mean','std']).reset_index()
combidataA.rename(columns={data.iloc[:,0].name:"unique_vals"},inplace=True)
combidataA.insert(0,"CFG_name",data.iloc[:,0].name,True)

"""
for i in (1,count_CFG):
  combidatax=data.groupby([data.iloc[:,i].name],as_index=False)[pernameslist].agg(['count','mean','std']).reset_index()
  ext=[data.iloc[:,i].name,uniqval,]

combidataA.rename(columns={data.iloc[:,0].name:"unique_vals"},inplace=True)
combidataA.insert(0,"CFG_name",data.iloc[:,0].name,True)
combidataA
"""

for i in range(1,count_CFG):
  #print("---cfg",i)
    combidataTemp=data.groupby([data.iloc[:,i].name],as_index=False)[pernameslist].agg(['count','mean','std']).reset_index()
    combidataTemp.rename(columns={data.iloc[:,i].name:"unique_vals"},inplace=True)
    combidataTemp.insert(0,"CFG_name",data.iloc[:,i].name,True)
    combidataA=pd.concat([combidataA,combidataTemp],axis=0)

combidataA.to_csv("hypo/final/singlegroupby.csv")
#combidataA['CFG_name', '']


In [6]:
#define number of process
"""
pr=6


import multiprocessing
import os 
pool = multiprocessing.Pool(processes=pr)
from functools import partial

"""

'\npr=6\n\n\nimport multiprocessing\nimport os \npool = multiprocessing.Pool(processes=pr)\nfrom functools import partial\n\n'

In [ ]:

remain=count_CFG%pr
bm=count_CFG-remain
iterable=[]
comdif=bm//pr
for i in range(pr+1):
  temp=i*comdif
  temp=int(temp)
  iterable.extend([temp])


def f(count_CFG,pernameslist,data,comdif,n):
  
  pid=os.getpid()
  combipair=data.groupby([data.iloc[:,n].name,data.iloc[:,n+1].name],as_index=False)[pernameslist].agg(['count','mean','std']).reset_index()

  combipair.rename(columns={data.iloc[:,n].name:"unique_vals1",data.iloc[:,n+1].name:"unique_vals2"},inplace=True)
  combipair.insert(0,"CFG1_name",data.iloc[:,n].name,True)
  combipair.insert(1,"CFG2_name",data.iloc[:,n+1].name,True)
  combipair=combipair.head(0)
  for i in range(n,n+comdif):
    for j in range(i+1,count_CFG):
      print("--CFG1 : ",i,"  | --CFG2 : ",j,"---")


      combipairTemp=data.groupby([data.iloc[:,i].name,data.iloc[:,j].name],as_index=False)[pernameslist].agg(['count','mean','std']).reset_index()
      combipairTemp.rename(columns={data.iloc[:,i].name:"unique_vals1",data.iloc[:,j].name:"unique_vals2"},inplace=True)
      combipairTemp.insert(0,"CFG1_name",data.iloc[:,i].name,True)
      combipairTemp.insert(1,"CFG2_name",data.iloc[:,j].name,True)
      combipair=pd.concat([combipair,combipairTemp],axis=0)
  
  filename="paircal"+str(pid)+".csv"
  path="hypo/"+filename
  combipair.to_csv(path)




#iterable = [0, 250, 500, 750,1000,1250]
#iterable = [0, 4, 8, 12,16,20]
#iterable = [0,1,2,3]



print(iterable)


func = partial(f, count_CFG,pernameslist,data,comdif)
pool.map(func, iterable)
pool.close()
pool.join()



[0, 231, 462, 693, 924, 1155, 1386]


In [3]:
import glob

path = 'hypo' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

fulldata = pd.concat(li, axis=0, ignore_index=True)
fulldata.dropna(inplace=True)
fulldata.to_csv("hypo/final/pairgroupbycal.csv")



KeyboardInterrupt: 

In [53]:
del(fulldata)

In [5]:
oricfgsingdf=pd.read_csv("hypo/final/singlegroupby.csv")
oricfgpairdf=pd.read_csv("hypo/final/pairgroupbycal.csv")
oricfgpairdf.drop(0, inplace=True)
oricfgsingdf.drop(0, inplace=True)
oricfgpairdf.dropna(inplace=True)
rowdf=data.copy()


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:

cfgsingdf=oricfgsingdf.copy()
cfgpairdf=oricfgpairdf.copy()



from itertools import islice
"""
for i in range(6,len(cfgpairdf.columns)):
  cfgpairdf.iloc[:,i]=int(cfgpairdf.iloc[:,i])
"""


#add columns to cfgsingdf to be equal to columns in cfgpairdf

#combipairTemp.insert(0,"CFG1_name",data.iloc[:,i].name,True)

cfgsingdf.insert(0,"testcol1",'testval',True)
cfgsingdf.insert(3,"testcol2",'0',True)
cfgsingdf.insert(5,"testcol3",'0',True)







#print(cfgsingdf)

output2d=[["CFG1_name","CFG_2name","CFG1_val","CFG2_val","PM_name","PM_Mean_of_column","PM_mean_for_CFG1_sample","PM_mean_for_CFG2_sample",
           "PM_mean_for_pair_sample","CFG1_count","CFG2_count","Pair_count","CFG1_std","CFG2_std","Pair_std","CFG1_zscore","CFG_2_zscore","pair_Z_score_withCFG1",
           "pair_Z_score_withCFG2","CFG1_Pvalue","CFG2_Pvalue","Pair_PvalueZscoreA","Pair_PvalueZscoreB","Pair_type"]]
#drop std null and 0 rows



#create PMvariable list ex- [PM_1 count, PM_1 mean, PM_1 std, PM_2 count, PM_2 Mean....]

pmcaltotlist=[]
pmvariablelist=[]


#print(pmcaltotlist)
#print("pm variable list   : ",pmvariablelist)


for i in range (count_Per):
    
    pmcount=rowdf.iloc[:,count_CFG+i].count()
    pmtotalmean=rowdf.iloc[:,count_CFG+i].mean()
    pmtotalstd=rowdf.iloc[:,count_CFG+i].std()
    pmcaltotlist.extend([pmcount,pmtotalmean,pmtotalstd])
    pmvname=rowdf.iloc[:,count_CFG+i].name
    pmvcount=pmvname
    pmvmean=pmvname+'.1'
    pmvstd=pmvname+'.2'

    pmvariablelist.extend([pmvcount,pmvmean,pmvstd])




#create PM_ count,mean,std list
cfglist=cfgsingdf["CFG_name"].unique()

#print(pmvariablelist)





#cfgpairdf['unique_vals1'].astype(float, inplace=True)
#cfgpairdf['unique_vals2'].astype(float, inplace=True)



cfgsingdf[pmvariablelist]=cfgsingdf[pmvariablelist].apply(pd.to_numeric)
cfgsingdf=cfgsingdf.round(3)
cfgpairdf=cfgpairdf.round(3)



#for index, row in islice(df.iterrows(), 1, None):
#1136154

#for i,row in islice(cfgpairdf.iterrows(),0,None):

for i in range(2300001,2480198):
    
    row=cfgpairdf.iloc[i]
    
    cfgAname=row["CFG1_name"]
    cfgBname=row["CFG2_name"]
    
    if ((cfgAname!='CFG_RruJointCalParaCfg.TxChnCalTime') and (cfgBname!='CFG_RruJointCalParaCfg.TxChnCalTime') and (cfgAname!='CFG_InterRatCellShutdown.StopTime') and (cfgBname!='CFG_InterRatCellShutdown.StopTime') and (cfgAname!='CFG_InterRatCellShutdown.StartTime') and (cfgBname!='CFG_InterRatCellShutdown.StartTime') and (cfgAname!='CFG_CellShutdown.StopTime') and (cfgBname!='CFG_CellShutdown.StopTime') and (cfgAname!='CFG_CellShutdown.StartTime') and (cfgBname!='CFG_CellShutdown.StartTime') and (cfgAname!='CFG_EutranInterFreqNCell.NCellAdditionTime') and (cfgBname!='CFG_EutranInterFreqNCell.NCellAdditionTime') and (cfgAname!='CFG_CellMBMSCfg.ServiceAreaIdList') and (cfgBname!='CFG_CellMBMSCfg.ServiceAreaIdList') and (cfgAname!='CFG_NbCellDynPwrShrPara.EndTime') and (cfgBname!='CFG_NbCellDynPwrShrPara.EndTime') and (cfgAname!='CFG_NbCellDynPwrShrPara.StartTime') and (cfgBname!='CFG_CellAccess.ReptSyncAvoidTime') and (cfgBname!='CFG_NbCellDynPwrShrPara.StartTime') and (cfgAname!='CFG_CellAccess.ReptSyncAvoidTime') and (cfgAname!='CFG_CellRfShutdown.StartTime') and (cfgBname!='CFG_CellRfShutdown.StopTime') and (cfgBname!='CFG_CellRfShutdown.StartTime') and (cfgAname!='CFG_CellRfShutdown.StopTime') and (cfgBname!='CFG_Cell.CellSlaveBand') and (cfgAname!='CFG_Cell.CellSlaveBand') and (cfgAname!='CFG_CellRfShutdown.StartTime') and (cfgBname!='CFG_CellAccess.RoundActionTime') and (cfgBname!='CFG_CellRfShutdown.StartTime') and (cfgAname!='CFG_CellAccess.RoundActionTime')):
    
        cfgAuniq=int(row["unique_vals1"])
        cfgAuniq=str(cfgAuniq)
        cfgBuniq=row["unique_vals2"]

        #print("CFGA and B names   :   ",cfgAname,cfgAuniq,cfgBname,cfgBuniq)
      #cfgAsingrow=cfgsingdf[cfgsingdf["CFG_name"]==cfgAname & cfgAuniq==["uniq_vals"]]
      #idx1 = np.where((cfgsingdf['CFG_name']==cfgAname.value) &amp; (cfgsingdf['unique_vals1']== cfgAuniq.value))

      #cfgAsingrow=cfgsingdf[idx1]
      #print(cfgsingdf['CFG_name']==cfgAname)&(cfgsingdf['unique_vals'==cfgAuniq])
        #cfgAtest=cfgsingdf[(cfgsingdf['CFG_name']==cfgAname)]
        #cfgA1singrow=cfgAtest[(cfgAtest['unique_vals']==cfgAuniq)]
        cfgAsingrow=cfgsingdf[(cfgsingdf['CFG_name']==cfgAname)&(cfgsingdf['unique_vals']==cfgAuniq)]
        cfgBsingrow=cfgsingdf[(cfgsingdf['CFG_name']==cfgBname)&(cfgsingdf['unique_vals']==cfgBuniq)]
        #print(i,"CFGA test row   ",cfgAtest)
        if (i%1000==0):
            print(i,"  pair row of 2480198 ")
      #print(i,"CFGB single row   ",cfgBsingrow)

      #idx2 = np.where((cfgsingdf["CFG_name"]==cfgBname) &amp; (cfgsingdf['uniq_vals2']== cfgBuniq))
      #cfgBsingrow=cfgsingdf[idx2]




      #cfgBsingrow=cfgsingdf[cfgsingdf["CFG_name"]==cfgBname & cfgBuniq["uniq_vals"]]



        for j in range(0,len(pmvariablelist),3):
            cfgAcount=cfgAsingrow[pmvariablelist[j]].mean() #pmvariablelist hold the names of pmcolomns
            cfgBcount=cfgBsingrow[pmvariablelist[j]].mean()
            pairABcount=float(row[pmvariablelist[j]])

            cfgAmean=cfgAsingrow[pmvariablelist[j+1]].mean()
            cfgBmean=cfgBsingrow[pmvariablelist[j+1]].mean()
            pairABmean=float(row[pmvariablelist[j+1]])

            cfgAstd=cfgAsingrow[pmvariablelist[j+2]].mean()
            cfgBstd=cfgBsingrow[pmvariablelist[j+2]].mean()
            pairABstd=float(cfgBsingrow[pmvariablelist[j+2]].mean())

            if ((cfgAstd!=0) and (cfgBstd!=0) and (pairABstd!=0)):

                #print("xox-xox")

                totmean=pmcaltotlist[j+1]
            #print("A")
                totstd=pmcaltotlist[j+2]
            #print("B")
                totcount=pmcaltotlist[j]
            #print("C")



                cfgAzscore=(totmean-cfgAmean)/(totstd/math.sqrt(totcount))
            #print("D")
                cfgBzscore=(totmean-cfgBmean)/(totstd/math.sqrt(totcount))
            #print("E")
                if (cfgAzscore<0):
                    cfgAzscore=-1*cfgAzscore  # -zscore * -1
                if (cfgBzscore<0):
                    cfgBzscore=-1*cfgBzscore




            #clculate Pvals

            #uniqcfg1pval=scipy.stats.norm.sf(abs(uniqcfg1zscore))
                cfgApval=scipy.stats.norm.sf(abs(cfgAzscore))
                cfgBpval=scipy.stats.norm.sf(abs(cfgBzscore))



            #calculate combination PVals

            #combi z score

                pairzscoreA=(cfgAmean-pairABmean)/(cfgAstd/math.sqrt(cfgAcount))
                pairzscoreB=(cfgBmean-pairABmean)/(cfgBstd/math.sqrt(cfgBcount))

                if(pairzscoreA<0):
                    pairzscoreA=-1*pairzscoreA

                if(pairzscoreB<0):
                    pairzscoreB=-1*pairzscoreB

                cfgABpval=scipy.stats.norm.sf(abs(pairzscoreA))
                cfgBApval=scipy.stats.norm.sf(abs(pairzscoreB))

                if ((cfgAcount==pairABcount) & (cfgAcount==cfgBcount)):
                    cfgtype='CFG1_CFG2_Perfect_Sub'
                elif((cfgAcount==cfgBcount) & (cfgBcount>pairABcount)):
                    cfgtype='CFG2_PartiallyIn_CFG1'
                elif((cfgAcount>cfgBcount) & (cfgBcount==pairABcount)):
                    cfgtype='CFG2_In_CFG1'
                elif((cfgAcount>cfgBcount) & (cfgBcount>pairABcount)):
                    cfgtype='CFG2_PartiallyIn_CFG1'
                elif((cfgAcount<cfgBcount) & (cfgAcount==pairABcount)):
                    cfgtype='CFG1_In_CFG2'
                elif((cfgAcount<cfgBcount) & (cfgAcount>pairABcount)):
                    cfgtype='CFG2_PartiallyIn_CFG1'
                else:
                    cfgtype='Unidentfied_Conection'


                rowline=[cfgAname,cfgBname,cfgAuniq,cfgBuniq,pmvariablelist[j],totmean,cfgAmean,cfgBmean,pairABmean,cfgAcount,cfgBcount,pairABcount,cfgAstd,cfgBstd,pairABstd,
                     cfgAzscore,cfgBzscore,pairzscoreA,pairzscoreB,cfgApval,cfgBpval,cfgABpval,cfgBApval,cfgtype]

                output2d.append(rowline)
"""
                if (i%100==0):
                    outputA=output2d.copy()
                    print(i,"th output2d   :::  ",output2d)
                    print(i,"th outputA   :::  ",outputA)
                    outputname= str(i)+'PvaluesPairs.csv'
                    with open(outputname,'w',newline='') as myoutput:
                        writer=csv.writer(myoutput)
                        writer.writerows(outputA)
                    
                    output2d=[["CFG1_name","CFG_2name","CFG1_val","CFG2_val","PM_name","PM_Mean_of_column","PM_mean_for_CFG1_sample","PM_mean_for_CFG2_sample",
                                        "PM_mean_for_pair_sample","CFG1_count","CFG2_count","Pair_count","CFG1_std","CFG2_std","Pair_std","CFG1_zscore","CFG_2_zscore","pair_Z_score_withCFG1",
                                        "pair_Z_score_withCFG2","CFG1_Pvalue","CFG2_Pvalue","Pair_PvalueZscoreA","Pair_PvalueZscoreB","Pair_type"]]


"""


    

    
with open('24milPvaluesPairs.csv','w',newline='') as myoutput:
    writer=csv.writer(myoutput)
    writer.writerows(output2d)








     
   






2301000   pair row of 2480198 
2302000   pair row of 2480198 
2303000   pair row of 2480198 
2304000   pair row of 2480198 
2305000   pair row of 2480198 
2306000   pair row of 2480198 
2307000   pair row of 2480198 
2308000   pair row of 2480198 
2309000   pair row of 2480198 
2310000   pair row of 2480198 
2311000   pair row of 2480198 
2312000   pair row of 2480198 
2313000   pair row of 2480198 
2314000   pair row of 2480198 
2315000   pair row of 2480198 
2316000   pair row of 2480198 
2317000   pair row of 2480198 
2318000   pair row of 2480198 
2319000   pair row of 2480198 
2321000   pair row of 2480198 
2322000   pair row of 2480198 
2323000   pair row of 2480198 
2324000   pair row of 2480198 
2325000   pair row of 2480198 
2326000   pair row of 2480198 
2327000   pair row of 2480198 
2328000   pair row of 2480198 
2329000   pair row of 2480198 
2330000   pair row of 2480198 
2331000   pair row of 2480198 
2332000   pair row of 2480198 
2333000   pair row of 2480198 
2334000 

ValueError: invalid literal for int() with base 10: 'LTE FDD'

In [7]:
i

2480195

In [8]:
cfgAname

'CFG_tech_desc'

In [9]:
with open('24milPvaluesPairs.csv','w',newline='') as myoutput:
    writer=csv.writer(myoutput)
    writer.writerows(output2d)

In [12]:
cfgpairdf.iloc[[10]]

,Unnamed: 0,Unnamed: 0.1,CFG1_name,CFG2_name,unique_vals1,unique_vals2,PM_L.UL.Interference.Avg(dBm),PM_L.UL.Interference.Avg(dBm).1,PM_L.UL.Interference.Avg(dBm).2,PM_Ref.Avg.CQI(number),...,PM_Ref.L.Thrp.GBytes.UL(GB).2,PM_Ref.L.User.Thrp.Kbps.DL(kbit/s),PM_Ref.L.User.Thrp.Kbps.DL(kbit/s).1,PM_Ref.L.User.Thrp.Kbps.DL(kbit/s).2,PM_Ref.L.User.Thrp.Kbps.UL.FDD(kbit/s),PM_Ref.L.User.Thrp.Kbps.UL.FDD(kbit/s).1,PM_Ref.L.User.Thrp.Kbps.UL.FDD(kbit/s).2,PM_Ref.VoLTE.Traffic(Erl)(Erl),PM_Ref.VoLTE.Traffic(Erl)(Erl).1,PM_Ref.VoLTE.Traffic(Erl)(Erl).2
11,12,2.0,CFG_BcchCfg.ModifyPeriodCoeff,CFG_CellBackOff.RBPriMcsSelectTrigPrbThd,1,10,7,-106.143,2.34,7,...,0.285,7,3909.209,2955.707,7,1885.644,867.111,7,1113.607,685.354


In [8]:
x=output2d.copy()
x

[['CFG1_name',
  'CFG_2name',
  'CFG1_val',
  'CFG2_val',
  'PM_name',
  'PM_Mean_of_column',
  'PM_mean_for_CFG1_sample',
  'PM_mean_for_CFG2_sample',
  'PM_mean_for_pair_sample',
  'CFG1_count',
  'CFG2_count',
  'Pair_count',
  'CFG1_std',
  'CFG2_std',
  'Pair_std',
  'CFG1_zscore',
  'CFG_2_zscore',
  'pair_Z_score_withCFG1',
  'pair_Z_score_withCFG2',
  'CFG1_Pvalue',
  'CFG2_Pvalue',
  'Pair_PvalueZscoreA',
  'Pair_PvalueZscoreB',
  'Pair_type'],
 ['CFG_BcchCfg.ModifyPeriodCoeff',
  'CFG_BcchCfg.ModifyPeriodCoeffForNb',
  '0',
  '1',
  'PM_L.UL.Interference.Avg(dBm)',
  -113.02355669473941,
  -113.029,
  -113.021,
  -113.027,
  8186.0,
  8164.0,
  8157.0,
  4.978,
  4.982,
  4.982,
  0.09893572477532078,
  0.046469642058202185,
  0.03635054893126931,
  0.10881757043510505,
  0.4605946518867073,
  0.48146796503250877,
  0.48550142216509506,
  0.45667359369222893,
  'CFG2_PartiallyIn_CFG1'],
 ['CFG_BcchCfg.ModifyPeriodCoeff',
  'CFG_BcchCfg.ModifyPeriodCoeffForNb',
  '0',
  '1',
 

In [23]:
len(cfgpairdf)

2480198

In [10]:
df=pd.read_csv("24milPvaluesPairs.csv")

df.head(5)

,CFG1_name,CFG_2name,CFG1_val,CFG2_val,PM_name,PM_Mean_of_column,PM_mean_for_CFG1_sample,PM_mean_for_CFG2_sample,PM_mean_for_pair_sample,CFG1_count,...,Pair_std,CFG1_zscore,CFG_2_zscore,pair_Z_score_withCFG1,pair_Z_score_withCFG2,CFG1_Pvalue,CFG2_Pvalue,Pair_PvalueZscoreA,Pair_PvalueZscoreB,Pair_type
0,CFG_NsaDcMgmtConfig.UlDataSplitThreshold,CFG_ULZeroBufferZone.ULZeroBufZoInterRatB1Timer,1,30,PM_L.UL.Interference.Avg(dBm),-113.023557,-113.196,-113.021,-113.196,7346.0,...,4.982,3.134273,0.046470,0.0,3.173846,8.614032e-04,0.481468,0.5,7.521679e-04,CFG1_In_CFG2
1,CFG_NsaDcMgmtConfig.UlDataSplitThreshold,CFG_ULZeroBufferZone.ULZeroBufZoInterRatB1Timer,1,30,PM_Ref.Avg.CQI(number),9.029470,9.167,9.030,9.167,7346.0,...,1.671,7.451347,0.028707,0.0,7.407909,4.619594e-14,0.488549,0.5,6.415334e-14,CFG1_In_CFG2
2,CFG_NsaDcMgmtConfig.UlDataSplitThreshold,CFG_ULZeroBufferZone.ULZeroBufZoInterRatB1Timer,1,30,PM_Ref.L.TA.UE.Index_All_Approx,264.229813,289.927,265.106,289.927,7346.0,...,3472.510,0.671011,0.022879,0.0,0.645843,2.511067e-01,0.490873,0.5,2.591904e-01,CFG1_In_CFG2
3,CFG_NsaDcMgmtConfig.UlDataSplitThreshold,CFG_ULZeroBufferZone.ULZeroBufZoInterRatB1Timer,1,30,PM_Ref.HO.Out.Success.Rate,99.729680,99.740,99.730,99.740,7346.0,...,1.812,0.516401,0.015991,0.0,0.498647,3.027872e-01,0.493621,0.5,3.090140e-01,CFG1_In_CFG2
4,CFG_NsaDcMgmtConfig.UlDataSplitThreshold,CFG_ULZeroBufferZone.ULZeroBufZoInterRatB1Timer,1,30,PM_Ref.L.E-RAB.AbnormRel.Rate (ALL)(%),0.145599,0.146,0.146,0.146,7346.0,...,0.207,0.175376,0.175376,0.0,0.000000,4.303919e-01,0.430392,0.5,5.000000e-01,CFG1_In_CFG2


In [12]:
df.dtypes

CFG1_name                   object
CFG_2name                   object
CFG1_val                     int64
CFG2_val                    object
PM_name                     object
PM_Mean_of_column          float64
PM_mean_for_CFG1_sample    float64
PM_mean_for_CFG2_sample    float64
PM_mean_for_pair_sample    float64
CFG1_count                 float64
CFG2_count                 float64
Pair_count                 float64
CFG1_std                   float64
CFG2_std                   float64
Pair_std                   float64
CFG1_zscore                float64
CFG_2_zscore               float64
pair_Z_score_withCFG1      float64
pair_Z_score_withCFG2      float64
CFG1_Pvalue                float64
CFG2_Pvalue                float64
Pair_PvalueZscoreA         float64
Pair_PvalueZscoreB         float64
Pair_type                   object
dtype: object

In [13]:
df.columns

Index(['CFG1_name', 'CFG_2name', 'CFG1_val', 'CFG2_val', 'PM_name',
       'PM_Mean_of_column', 'PM_mean_for_CFG1_sample',
       'PM_mean_for_CFG2_sample', 'PM_mean_for_pair_sample', 'CFG1_count',
       'CFG2_count', 'Pair_count', 'CFG1_std', 'CFG2_std', 'Pair_std',
       'CFG1_zscore', 'CFG_2_zscore', 'pair_Z_score_withCFG1',
       'pair_Z_score_withCFG2', 'CFG1_Pvalue', 'CFG2_Pvalue',
       'Pair_PvalueZscoreA', 'Pair_PvalueZscoreB', 'Pair_type'],
      dtype='object')